In [2]:
import xarray as xr
import numpy as np
from scipy import stats
import os

os.chdir("/gws/nopw/j04/aopp/jessew/stuff_for_jesse") # go back to parent directory

In [4]:
faulty_observed_delta_SST = xr.open_dataarray('CNN_models/observed_delta_SST_1980-99.nc').compute() #from -60 to 60
observed_SST = xr.open_dataarray("CNN_models/hadsst_annual_SSTgrid.nc").sel(lat=slice(60, -60)).compute()

In [75]:
observed_delta_SST = observed_SST - observed_SST.sel(time=slice('1980', '1999')).mean('time')

In [76]:
precip_lat = -55.0
precip_lon = 95.0

lat_filename = str(format(precip_lat, '.1f')).replace(".","")
lon_filename = str(format(precip_lon, '.1f')).replace(".","")
gto_filepath = "GTO_data_10deg_train/GTO_lat{}_lon{}.nc".format(lat_filename, lon_filename)
G = xr.open_dataarray(gto_filepath)
G = G.sel(lat=observed_delta_SST.lat)

weights = np.cos(np.deg2rad(G.lat))* 6371**2 * 1 * 1 * (np.pi/180)**2 * G
# Okay up until here...

product = observed_delta_SST.weighted(weights.fillna(0))

# bad here.
prediction = product.sum(('lon', 'lat'), skipna=True)
                
observed_precip = xr.open_dataarray("observed_mean_annual.nc").compute()
P_tot = observed_precip.sel(lat=precip_lat, lon=precip_lon).values

In [113]:
product = observed_delta_SST.dot(weights.fillna(0))

In [114]:
product = product.values.reshape(1,-1)

In [115]:
np.concatenate((product, product), axis=0).shape

(2, 149)

In [8]:
lats = observed_SST.lat
lons = observed_SST.lon

In [28]:
G = xr.DataArray(np.ones((len(lats),len(lons))), coords=[lats, lons], dims=["lat", "lon"]) 
C = xr.DataArray(np.zeros((len(lats),len(lons))), coords=[lats, lons], dims=["lat", "lon"]) 
B = xr.DataArray(np.ones((len(lats),len(lons))), coords=[lats, lons], dims=["lat", "lon"]) 

In [29]:
new_array = xr.concat([G, C, B], 'abcg_dim')

In [33]:
new_array[1]

<xarray.DataArray (lat: 97, lon: 192)>
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])
Coordinates:
  * lat      (lat) float32 60.0 58.75 57.5 56.25 ... -56.25 -57.5 -58.75 -60.0
  * lon      (lon) float32 0.0 1.875 3.75 5.625 7.5 ... 352.5 354.4 356.2 358.1